In [1]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

In [4]:
DATA_STORE = Path('assets.h5')

In [13]:
import pandas as pd
import gdown
import os

# File path
file_path = 'wiki_prices.csv'

# Download the file from Google Drive only if it doesn't exist
if not os.path.exists(file_path):
    gdown.download('https://drive.google.com/uc?export=download&id=1RTEvR3S_jNZNIn_WldbCKfsDb2pl3KZf', file_path, quiet=False)
else:
    print(f"File '{file_path}' already exists. Skipping download.")

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path,
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True).sort_index()

# Now you can work with the DataFrame 'df'


print(df.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)

File 'wiki_prices.csv' already exists. Skipping download.
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   open         15388776 non-null  float64
 1   high         15389259 non-null  float64
 2   low          15389259 non-null  float64
 3   close        15389313 non-null  float64
 4   volume       15389314 non-null  float64
 5   ex-dividend  15389314 non-null  float64
 6   split_ratio  15389313 non-null  float64
 7   adj_open     15388776 non-null  float64
 8   adj_high     15389259 non-null  float64
 9   adj_low      15389259 non-null  float64
 10  adj_close    15389313 non-null  float64
 11  adj_volume   15389314 non-null  float64
dtypes: float64(12)
memory usage: 1.4+ GB
None


ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.